In [5]:
import pandas as pd
import numpy as np
# 数据加载
C_df = pd.read_excel("data\\Concentration in Biota.xlsx") # 各国家各鱼体内PFOS浓度
P_df =pd.read_excel("data\\Production.xlsx",index_col=0).fillna(0) # 各国家各鱼类的产量
print(P_df.shape)
P_df.head()
# P_df.isna().sum()
code_df =  pd.read_excel("data\\Meaning of Code.xlsx") # 每个代码包含的具体鱼种
code_df.head()

# 变量初始化
chem_df_lis = []

code_lis = code_df.columns
print(code_lis)
C_fami = ['mean','p025','p975']
chem_lis = np.unique(C_df.Chemical)

pop_df = pd.read_excel("data\\Population.xlsx",index_col=0,names=['pop'])
#country_lis = np.unique(C_df.Country.to_list())
country_lis = pop_df.index.to_list()
print(chem_lis)
print(chem_lis.size)
print(len(country_lis))

(33, 212)
Index([30194, 30195, 30212, 30213, 30214, 30219, 30221, 30222, 30223, 30224,
       ...
       30481, 30483, 30484, 30486, 30487, 30488, 30491, 30495, 30496, 30497],
      dtype='int64', length=111)
['PFBS' 'PFDA' 'PFHpA' 'PFHxA' 'PFHxS' 'PFNA' 'PFOA' 'PFOS' 'PFUnDA']
9
33


In [6]:
## 工具函数
### 格式初始化

def init_data(code_lis=code_lis):
    """
    初始化数据字典，格式为{(鱼类代码, 浓度类型): [国家浓度列表]}。
    用于存储每个鱼类和浓度类型对应的各国浓度数据。
    """
    dic = {}
    for code in code_lis:
        for C_fam in C_fami:
            dic[(code,C_fam)] = list([])
    return dic


In [7]:
from tqdm import tqdm
import warnings
# 禁用所有警告
warnings.filterwarnings("ignore")

def check_data_integrity(_country, _fish, p_df,_code):
    """
    检查数据完整性，确保查询的浓度数据和产量数据是有效的。
    若发现问题，打印警告信息。
    
    Args:
        _country (str): 国家名称
        _fish (str): 鱼种名称
        p_df (DataFrame): 产量数据
    
    Returns:
        bool: 如果数据有效则返回True，若有问题返回False
    """
    # 检查产量数据是否存在
    try:
        _p = p_df.loc[_country, _fish]  # 获取产量数据
    except KeyError:
        raise KeyError(f"警告: {_country}的{_fish}在产量数据Production中不存在!  该鱼类属于code{_code}")
              

def get_chem_country(_chem) -> list:
    """
    根据化学物质名称计算每个国家每个代码对应的浓度。

    Args:
        _chem (str): 化学物质的名称，用于查找相关数据。

    Returns:
        dict: 返回一个字典，格式为{(鱼类代码, 浓度类型): [国家浓度列表]}。
    """
    data_dic = init_data()
    # 先预处理：设置索引和过滤相关的化学物质数据，避免在循环中重复操作
    _c_df = C_df.set_index(['Country', 'Species', 'Chemical'])  # 统一设置索引
    
    for _code in tqdm(code_lis):
        fish_in_code = list(code_df[_code].dropna())
        
        # 遍历浓度类型
        for c_type in C_fami:
            # country_lis = []
            _lis=[]
            for _country in country_lis:
                c_country_lis = []
                p_country_lis = []
                
                # 获取该国家的所有鱼类浓度数据
                _c_df_c = _c_df.loc[(_country)]
                
                for _fish in fish_in_code:
                    check_data_integrity(_country, _fish, P_df,_code)
                    
                    _p = P_df.loc[_country]
                    _p = _p[_fish] # 产量数据
                    
                    _flag = True
                    
                    # 如果该鱼种在该化学物质下没有浓度数据
                    if (_fish,_chem) not in _c_df_c.index:
                        _c= 0
                        _flag = False
                    else:
                        _c=float(_c_df.loc[(_country,_fish,_chem)][c_type])
                    c_country_lis.append(_c)
                    p_country_lis.append(_p)
                    
                    if _c ==0 and _p !=0 and _flag:
                        print(f"{_country}的{_fish}捕捞量为{_p},但是{_chem}浓度c为{_c}")
                # 计算加权平均浓度
                c_country_arr = np.array(c_country_lis)  # 浓度值数组
                p_country_arr = np.array(p_country_lis)  # 产量值数组
                _cp = np.sum(c_country_arr * p_country_arr)
                _p_sum = np.sum(p_country_arr)  # 产量总和

                # 计算加权平均浓度，若总产量为0则返回0
                if _p_sum != 0:
                    _lis.append(_cp / _p_sum)
                else:
                    _lis.append(0)
            
            data_dic[(_code,c_type)] = _lis
    return data_dic
#dic = get_chem_country(_chem)
# print(dic)

In [8]:
# 创建一个ExcelWriter对象，指定文件名
excel_writer = pd.ExcelWriter('Concentration_corresponding_to_the_code.xlsx')

# 将DataFrame写入Excel文件的不同工作表
for _chem in chem_lis:
    dic = get_chem_country(_chem)
    df = pd.DataFrame(dic,index=country_lis)
    df.to_excel(excel_writer,sheet_name=_chem,index=True)

excel_writer.close()

100%|██████████| 111/111 [00:02<00:00, 46.86it/s]


In [9]:
family_df = pd.read_excel("data\Family-Species.xlsx") # 每科对应的鱼种类

def get_chem_country_family(_chem)->dict:
    """
    计算每个国家每个鱼类科在给定化学物质（_chem）下的浓度，加权后生成数据。
    Args:
        _chem (str): 需要计算的化学物质名称。
    Returns:
        dict: {(鱼类代码, 浓度类型): [国家浓度列表]}。
    """
    data_dic = init_data(code_lis=family_df.columns)
    for _code in tqdm(family_df.columns):
        fish_in_code = list(family_df[_code].dropna())
        for c_type in C_fami:
            _lis=[]
            for _country in country_lis:
                c_country_lis = []
                p_country_lis = []
                
                _c_df = C_df.copy()
                _c_df.set_index([ 'Country','Species','Chemical'],inplace=True)
                _c_df_c = _c_df.loc[(_country)]
                for _fish in fish_in_code:
                    _c = 0
                    _p = P_df.loc[_country][_fish]
                    _flag = True
                    
                    if (_fish,_chem) not in _c_df_c.index:
                        _c= 0
                        _flag = False
                    else:
                        _c=float(_c_df.loc[(_country,_fish,_chem)][c_type])
                    
                    c_country_lis.append(_c)
                    p_country_lis.append(_p)
                    
                    if _c ==0 and _p !=0 and _flag:
                        print(f"{_country}的{_fish}捕捞量为{_p},但是{_chem}浓度c为{_c}")
                
                _cp=0
                _p_sum =0
                for i in range(len(c_country_lis)):
                    _cp += c_country_lis[i]*p_country_lis[i]
                    if _cp !=0:
                        _p_sum += p_country_lis[i]
                    
                if _p_sum !=0:
                    _lis.append(_cp)
                else:
                    _lis.append(0)
            
            data_dic[(_code,c_type)] = _lis
    return data_dic

In [10]:
# 创建一个ExcelWriter对象，指定文件名
excel_writer = pd.ExcelWriter('Country-Family-Weight.xlsx', engine='xlsxwriter')

# 将DataFrame写入Excel文件的不同工作表
for _chem in chem_lis:
    dic = get_chem_country_family(_chem)
    df = pd.DataFrame(dic,index=country_lis)
    df.to_excel(excel_writer,sheet_name=_chem,index=True)

excel_writer.close()


100%|██████████| 10/10 [00:02<00:00,  4.98it/s]


In [11]:
dic = get_chem_country_family(_chem)
dic.keys()

100%|██████████| 10/10 [00:02<00:00,  4.95it/s]


dict_keys([('Anchovies', 'mean'), ('Anchovies', 'p025'), ('Anchovies', 'p975'), ('Cod-likes', 'mean'), ('Cod-likes', 'p025'), ('Cod-likes', 'p975'), ('Flatfishes', 'mean'), ('Flatfishes', 'p025'), ('Flatfishes', 'p975'), ('Herring-likes', 'mean'), ('Herring-likes', 'p025'), ('Herring-likes', 'p975'), ('Perch-likes', 'mean'), ('Perch-likes', 'p025'), ('Perch-likes', 'p975'), ('Salmon, smelts, etc', 'mean'), ('Salmon, smelts, etc', 'p025'), ('Salmon, smelts, etc', 'p975'), ('Scorpionfishes', 'mean'), ('Scorpionfishes', 'p025'), ('Scorpionfishes', 'p975'), ('Sharks & rays', 'mean'), ('Sharks & rays', 'p025'), ('Sharks & rays', 'p975'), ('Tuna & billfishes', 'mean'), ('Tuna & billfishes', 'p025'), ('Tuna & billfishes', 'p975'), ('Other fishes & inverts', 'mean'), ('Other fishes & inverts', 'p025'), ('Other fishes & inverts', 'p975')])

In [12]:
family_df = pd.read_excel("data\Family-Species.xlsx")
family_df.columns

Index(['Anchovies', 'Cod-likes', 'Flatfishes', 'Herring-likes', 'Perch-likes',
       'Salmon, smelts, etc', 'Scorpionfishes', 'Sharks & rays',
       'Tuna & billfishes', 'Other fishes & inverts'],
      dtype='object')

In [13]:
################################

In [14]:
##############################

In [15]:
#################################

In [16]:
C_c_code = pd.read_excel("Concentration_corresponding_to_the_code.xlsx",index_col=0,header=[0,1])
print(C_c_code.shape)
C_c_code.head()

(33, 333)


30194                           30195                        \
                 mean       p025       p975      mean       p025       p975   
Australia   38.949675  38.759733  39.139624  22.18899  22.001469  22.379988   
Bangladesh   0.000000   0.000000   0.000000   0.00000   0.000000   0.000000   
Brazil       0.000000   0.000000   0.000000   0.00000   0.000000   0.000000   
Canada       1.741968   1.733473   1.750463   0.00000   0.000000   0.000000   
China        6.841739   6.808374   6.875104   0.00000   0.000000   0.000000   

               30212                        30213  ...      30491      30495  \
                mean      p025      p975     mean  ...       p975       mean   
Australia   0.000000  0.000000  0.000000  0.00000  ...  11.004287  38.949675   
Bangladesh  0.000000  0.000000  0.000000  0.00000  ...   0.000000   0.000000   
Brazil      0.000000  0.000000  0.000000  0.00000  ...   5.880229   7.312821   
Canada      1.066937  1.061649  1.072228  0.32867  ...   1.353719   1.647703   
China       0.000000  0.000000  0.000000  0.00000  ...   0.647346  16.250635   

                                      30496                       30497       \
                 p025       p975       mean       p025       p975  mean p025   
Australia   38.759733  39.139624  39.424311  39.232054  39.616574   0.0  0.0   
Bangladesh   0.000000   0.000000   0.000000   0.000000   0.000000   0.0  0.0   
Brazil       7.277160   7.348484   6.030648   6.001239   6.060058   0.0  0.0   
Canada       1.639607   1.655803   1.620491   1.612588   1.628393   0.0  0.0   
China       16.171387  16.329886   0.844382   0.840264   0.848500   0.0  0.0   

                 
           p975  
Australia   0.0  
Bangladesh  0.0  
Brazil      0.0  
Canada      0.0  
China       0.0  

[5 rows x 333 columns]

In [17]:
p_c_ex_code = pd.read_excel("data\Export.xlsx",index_col=0).fillna(0) # 出口数据
print(p_c_ex_code.shape)
p_c_ex_code.head()

(33, 111)


,30194,30195,30212,30213,30214,30219,30221,30222,30223,30224,...,30481,30483,30484,30486,30487,30488,30491,30495,30496,30497
Australia,70904.5,0.0,0.0,191.4,10218060.83,11899.65,0.0,0.0,0.0,0.0,...,49231.97,50.0,0.0,0.0,1189.98,23000.0,34.4,1297.3,0.0,0.0
Bangladesh,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
Brazil,0.0,41.0,0.0,528.0,22056.00,4251.00,120.0,0.0,390.0,20.0,...,12174.00,1071.0,0.0,385.0,172244.00,12485.0,10.0,0.0,0.0,0.0
Canada,207748.0,0.0,0.0,2156815.0,77381333.00,10807.00,5812510.0,0.0,34766.0,15.0,...,1983887.00,5313476.0,5783.0,501218.0,733752.00,3275.0,0.0,1956667.0,0.0,22630.0
China,0.0,0.0,0.0,257525.0,757590.00,0.00,984.0,0.0,278168.0,60.0,...,85088293.00,63923864.0,42158.0,584542.0,12679669.00,2156385.0,682788.0,9917287.0,2050258.0,0.0


In [18]:
_p_lis = p_c_ex_code.index.to_list()
_c_lis = C_c_code.index.to_list()
for x in _p_lis:
    if x not in _c_lis:
        print(x)

In [19]:
C_fami

['mean', 'p025', 'p975']

In [20]:
C_c_code = pd.read_excel("Concentration_corresponding_to_the_code.xlsx",index_col=0,header=[0,1],
                            sheet_name='PFOS')
# C_c_code.loc[('Canada',('030213','mean'))]

In [21]:
excel_writer = pd.ExcelWriter('Export-Weight.xlsx', engine='xlsxwriter')


for _chem in chem_lis:
    C_c_code = pd.read_excel("Concentration_corresponding_to_the_code.xlsx",index_col=0,header=[0,1],
                            sheet_name=_chem)
    W_df = C_c_code.copy()
    for _code in C_c_code.columns.get_level_values(0).to_list():
        for _c_type in C_fami:
            C_arr = C_c_code[(_code,_c_type)]
            
            _lis=[]
            for i in range(len(C_arr)):
                _lis.append(C_arr[i]*p_c_ex_code.loc[C_c_code.index[i] ,_code])
            W_df[(_code,_c_type)] = _lis
    W_df.to_excel(excel_writer,sheet_name=_chem,index=True)
excel_writer.close()

In [22]:
################################
##############################
#################################

In [23]:

import_df = pd.read_excel('data\Import.xlsx',index_col=0,sheet_name='Canada').fillna(0)
import_df.head(1)

,30194,30195,30212,30213,30214,30219,30221,30222,30223,30224,...,30481,30483,30484,30486,30487,30488,30491,30495,30496,30497
Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
C_c_code = pd.read_excel("Concentration_corresponding_to_the_code.xlsx",
                        index_col=0,header=[0,1],
                        sheet_name='PFOS')
C_c_code.head()

30194                              30195              \
                  mean        p025        p975       mean        p025   
Australia   962.154367  935.686972  988.175885  534.77909  509.166993   
Bangladesh    0.000000    0.000000    0.000000    0.00000    0.000000   
Brazil        0.000000    0.000000    0.000000    0.00000    0.000000   
Canada      105.159536  102.266758  108.003581    0.00000    0.000000   
China       319.341640  310.557040  327.978252    0.00000    0.000000   

                             30212                               30213  ...  \
                  p975        mean        p025        p975        mean  ...   
Australia   559.911806    0.000000    0.000000    0.000000    0.000000  ...   
Bangladesh    0.000000    0.000000    0.000000    0.000000    0.000000  ...   
Brazil        0.000000    0.000000    0.000000    0.000000    0.000000  ...   
Canada        0.000000  251.274392  243.784558  258.635512  403.247725  ...   
China         0.000000    0.000000    0.000000    0.000000    0.000000  ...   

                 30491       30495                                30496  \
                  p975        mean        p025        p975         mean   
Australia   303.739532  962.154367  935.686972  988.175885  1000.955931   
Bangladesh    0.000000    0.000000    0.000000    0.000000     0.000000   
Brazil      258.968857  317.791515  309.049556  326.386204   260.150829   
Canada       76.351404  121.196644  117.458718  124.869825    95.676722   
China        18.793043  409.763036  398.491082  420.845100    29.053327   

                                    30497            
                  p025         p975  mean p025 p975  
Australia   973.421165  1028.026840   0.0  0.0  0.0  
Bangladesh    0.000000     0.000000   0.0  0.0  0.0  
Brazil      252.994478   267.186623   0.0  0.0  0.0  
Canada       93.044802    98.264304   0.0  0.0  0.0  
China        28.254115    29.839076   0.0  0.0  0.0  

[5 rows x 333 columns]

In [25]:
excel_writer = pd.ExcelWriter('Import-Weight.xlsx', engine='xlsxwriter')

for _c in country_lis:
    import_df = pd.read_excel('data\Import.xlsx',index_col=0,sheet_name=_c).fillna(0)
    
    _df = C_c_code.copy()
    # 假设你的DataFrame为C_c_code
    _df.drop(_c, axis=0, inplace=True)
    for _code in C_c_code.columns.get_level_values(0).to_list():
        
        for _c_type in C_fami:
            C_arr = _df[(_code,_c_type)]
            
            _lis=[]
            for i in range(len(C_arr)):
                if import_df.loc[_df.index[i],_code] =='/':
                    print(_c,_df.index[i],_code)
                #print(_df.index[i],_code)
                #print(import_df.loc[_df.index[i],_code])
                _lis.append(C_arr[i]*import_df.loc[_df.index[i],_code])
        
        _df[(_code,_c_type)] = _lis
    _df.to_excel(excel_writer,sheet_name=_c,index=True)
excel_writer.close()

In [26]:
from tqdm import tqdm
for _chem in tqdm(chem_lis):  
    excel_writer = pd.ExcelWriter(f'Import-Weight-{_chem}.xlsx', engine='xlsxwriter')
    for _c in country_lis:
        import_df = pd.read_excel('data\Import.xlsx',
                                index_col=0,
                                sheet_name=_c).fillna(0)
        
        C_c_code = pd.read_excel("Concentration_corresponding_to_the_code.xlsx",
                        index_col=0,header=[0,1],
                        sheet_name=_chem)
        _df = C_c_code.copy()
        
        _df.drop(_c, axis=0, inplace=True)
        for _code in C_c_code.columns.get_level_values(0).to_list():
            
            for _c_type in C_fami:
                __df = C_c_code.copy()
                __df.drop(_c, axis=0, inplace=True)
                C_arr = __df[(_code,_c_type)]
                
                _lis=[]
                for i in range(len(C_arr)):
                    if import_df.loc[_df.index[i],_code] =='/':
                        print(_c,_df.index[i],_code)
                        
                    _lis.append(C_arr[_df.index[i]]*import_df.loc[_df.index[i],_code])
            
                _df[(_code,_c_type)] = _lis
        _df.to_excel(excel_writer,sheet_name=_c,index=True)
    excel_writer.close()

100%|██████████| 9/9 [04:11<00:00, 27.89s/it]


In [27]:
################################
##############################
#################################

In [28]:
W_c_df =pd.read_excel('Country-Family-Weight.xlsx',index_col=0,header=[0,1])
W_c_df.head()

Anchovies                                 Cod-likes  \
                    mean          p025          p975          mean   
Australia   0.000000e+00  0.000000e+00  0.000000e+00  3.268985e+07   
Bangladesh  2.966034e+08  2.940968e+08  2.991565e+08  0.000000e+00   
Brazil      1.196255e+04  1.179077e+04  1.214324e+04  7.478971e+06   
Canada      0.000000e+00  0.000000e+00  0.000000e+00  7.075019e+07   
China       4.913486e+09  4.871962e+09  4.955780e+09  2.382880e+08   

                                          Flatfishes                \
                    p025          p975          mean          p025   
Australia   3.253043e+07  3.284927e+07  4.155012e+04  4.134749e+04   
Bangladesh  0.000000e+00  0.000000e+00  9.388107e+07  9.308767e+07   
Brazil      7.442499e+06  7.515444e+06  0.000000e+00  0.000000e+00   
Canada      7.040253e+07  7.109798e+07  1.743051e+07  1.734551e+07   
China       2.371259e+08  2.394500e+08  1.014391e+08  1.009444e+08   

                         Herring-likes  ... Scorpionfishes Sharks & rays  \
                    p975          mean  ...           p975          mean   
Australia   4.175275e+04  2.458817e+07  ...   2.319856e+06  1.271727e+07   
Bangladesh  9.468918e+07  1.590895e+09  ...   9.216305e+06  0.000000e+00   
Brazil      0.000000e+00  0.000000e+00  ...   0.000000e+00  3.406422e+07   
Canada      1.751551e+07  4.783173e+07  ...   0.000000e+00  1.396410e+05   
China       1.019338e+08  3.065110e+08  ...   4.298452e+07  8.023038e+05   

                                       Tuna & billfishes                \
                    p025          p975              mean          p025   
Australia   1.265525e+07  1.277929e+07      1.581497e+08  1.568929e+08   
Bangladesh  0.000000e+00  0.000000e+00      0.000000e+00  0.000000e+00   
Brazil      3.389811e+07  3.423035e+07      2.206070e+08  2.195312e+08   
Canada      1.389601e+05  1.403220e+05      2.098593e+06  2.088359e+06   
China       7.983913e+05  8.062165e+05      3.697424e+07  3.679393e+07   

                         Other fishes & inverts                              
                    p975                   mean          p025          p975  
Australia   1.594279e+08           2.761574e+07  2.748073e+07  2.775077e+07  
Bangladesh  0.000000e+00           3.021718e+09  3.006982e+09  3.036454e+09  
Brazil      2.216829e+08           6.344775e+07  6.313834e+07  6.375717e+07  
Canada      2.108827e+06           0.000000e+00  0.000000e+00  0.000000e+00  
China       3.715456e+07           1.096931e+10  1.089882e+10  1.104054e+10  

[5 rows x 30 columns]

In [29]:
W_c_E_code_df = pd.read_excel('Export-Weight.xlsx',index_col=0,header=[0,1])
W_c_E_code_df.head()

30194                             30195           30212  \
                    mean          p025          p975  mean p025 p975  mean   
Australia   2.761707e+06  2.748240e+06  2.775175e+06   0.0  0.0  0.0     0   
Bangladesh  0.000000e+00  0.000000e+00  0.000000e+00   0.0  0.0  0.0     0   
Brazil      0.000000e+00  0.000000e+00  0.000000e+00   0.0  0.0  0.0     0   
Canada      3.618903e+05  3.601255e+05  3.636551e+05   0.0  0.0  0.0     0   
China       0.000000e+00  0.000000e+00  0.000000e+00   0.0  0.0  0.0     0   

                              30213  ...          30491         30495  \
           p025 p975           mean  ...           p975          mean   
Australia     0    0       0.000000  ...     378.547472  5.052941e+04   
Bangladesh    0    0       0.000000  ...       0.000000  0.000000e+00   
Brazil        0    0       0.000000  ...      58.802292  0.000000e+00   
Canada        0    0  708881.164973  ...       0.000000  3.224007e+06   
China         0    0       0.000000  ...  441999.828722  1.611622e+08   

                                               30496                \
                    p025          p975          mean          p025   
Australia   5.028300e+04  5.077583e+04  0.000000e+00  0.000000e+00   
Bangladesh  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
Brazil      0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
Canada      3.208164e+06  3.239855e+06  0.000000e+00  0.000000e+00   
China       1.603763e+08  1.619482e+08  1.731201e+06  1.722758e+06   

                         30497            
                    p975  mean p025 p975  
Australia   0.000000e+00   0.0  0.0  0.0  
Bangladesh  0.000000e+00   0.0  0.0  0.0  
Brazil      0.000000e+00   0.0  0.0  0.0  
Canada      0.000000e+00   0.0  0.0  0.0  
China       1.739643e+06   0.0  0.0  0.0  

[5 rows x 333 columns]

In [30]:
_chem = 'PFOS'
W_c_A_Import_df = pd.read_excel(f'Import-Weight-{_chem}.xlsx',
                                index_col=0,header=[0,1])
W_c_A_Import_df.head()

30194           30195           30212           30213  ... 30491  \
            mean p025 p975  mean p025 p975  mean p025 p975  mean  ...  p975   
Bangladesh     0    0    0     0    0    0     0    0    0     0  ...   0.0   
Brazil         0    0    0     0    0    0     0    0    0     0  ...   0.0   
Canada         0    0    0     0    0    0     0    0    0     0  ...   0.0   
China          0    0    0     0    0    0     0    0    0     0  ...   0.0   
Colombia       0    0    0     0    0    0     0    0    0     0  ...   0.0   

                   30495                             30496           30497  \
                    mean          p025          p975  mean p025 p975  mean   
Bangladesh  0.000000e+00  0.000000e+00  0.000000e+00   0.0  0.0  0.0     0   
Brazil      0.000000e+00  0.000000e+00  0.000000e+00   0.0  0.0  0.0     0   
Canada      0.000000e+00  0.000000e+00  0.000000e+00   0.0  0.0  0.0     0   
China       3.036754e+07  2.953217e+07  3.118883e+07   0.0  0.0  0.0     0   
Colombia    0.000000e+00  0.000000e+00  0.000000e+00   0.0  0.0  0.0     0   

                      
           p025 p975  
Bangladesh    0    0  
Brazil        0    0  
Canada        0    0  
China         0    0  
Colombia      0    0  

[5 rows x 333 columns]

In [31]:
pop_df = pd.read_excel("data\Population.xlsx",index_col=0,names=['pop'])
pop_df.head()

,pop
Australia,25357170
Bangladesh,165516220
Brazil,211782880
Canada,37522580
China,1421864030


In [32]:
code_family = pd.read_excel("data\Code-Family.xlsx")
str(int(code_family.iloc[0,0]))
code_family.head()

,Anchovies,Cod-likes,Flatfishes,Herring-likes,Perch-likes,"Salmon, smelts, etc",Scorpionfishes,Sharks & rays,Tuna & billfishes,Other fishes & inverts
0,30242.0,30250.0,30221.0,30240.0,30244.0,30212.0,NaN,30265.0,30194,NaN
1,NaN,30251.0,30222.0,30241.0,30245.0,30213.0,NaN,30281.0,30195,NaN
2,NaN,30252.0,30223.0,30243.0,30246.0,30214.0,NaN,30282.0,30231,NaN
3,NaN,30253.0,30224.0,30261.0,30264.0,30219.0,NaN,30375.0,30232,NaN
4,NaN,30254.0,30229.0,30350.0,30284.0,30310.0,NaN,30381.0,30233,NaN


In [33]:
W_c_df =pd.read_excel('Country-Family-Weight.xlsx',index_col=0,header=[0,1],
                        sheet_name=_chem)
np.unique(W_c_df.columns.get_level_values(0).to_list())

array(['Anchovies', 'Cod-likes', 'Flatfishes', 'Herring-likes',
       'Other fishes & inverts', 'Perch-likes', 'Salmon, smelts, etc',
       'Scorpionfishes', 'Sharks & rays', 'Tuna & billfishes'],
      dtype='<U22')

In [34]:
from tqdm import tqdm
excel_writer = pd.ExcelWriter('EDI-Family.xlsx', engine='xlsxwriter')
for _chem in chem_lis: 
    code_family = pd.read_excel("data\Code-Family.xlsx")
    W_c_df =pd.read_excel('Country-Family-Weight.xlsx',index_col=0,header=[0,1],
                        sheet_name=_chem)
    W_c_E_code_df = pd.read_excel('Export-Weight.xlsx',index_col=0,header=[0,1],
                                sheet_name=_chem)
    pop_df = pd.read_excel("data\Population.xlsx",index_col=0,names=['pop'])

    _df = W_c_df.copy()
    for _family in tqdm(np.unique(W_c_df.columns.get_level_values(0).to_list())):
        code_in_family = code_family[_family]
        for _c_type in C_fami:
            _W_c_arr = W_c_df[(_family,_c_type)]
            _lis = []
            
            for i in range(len(_df.index)):
                _W_c = _W_c_arr[i]
                
                if _family =='Salmonidae 鲑科' and _df.index[i] =='Canada' and _chem =='PFOA':
                    print(f'W_C_{_c_type} = {_W_c}')
                    
                for _code in code_in_family:
                    if np.isnan(_code):
                        continue
                    #_code = '0'+str(int(_code))
                    _we = W_c_E_code_df.loc[(_df.index[i],(_code,_c_type))]
                    _W_c -= _we
                    if _family =='Salmonidae 鲑科' and _df.index[i] =='Canada' \
                        and _chem =='PFOS' and _we!=0 and _c_type == C_fami[2]:
                        print(_code)
                        print(f'W_C_{_c_type}- {_we}  ={_W_c}')
                
                W_c_A_Import_df = pd.read_excel(f'Import-Weight-{_chem}.xlsx',
                        index_col=0,header=[0,1],
                        sheet_name=_df.index[i])
                
                for _code in code_in_family:
                    if np.isnan(_code):
                        continue
                    #_code = '0'+str(int(_code))
                    # print(_code)
                    for _x in W_c_A_Import_df[(_code,_c_type)]:
                        _W_c+=_x
                        if _family =='Salmonidae 鲑科' and \
                            _df.index[i] =='Canada' and _chem =='PFOA' and _x!=0 and _c_type == C_fami[2]:
                            print(_x)
                            print(f'W_C_{_c_type}+ {_x}  ={_W_c}')
                _pop= pop_df.loc[_df.index[i],'pop']
                
                Weight = 65 # 成年人体重kg
                if _family =='Salmonidae 鲑科' and \
                    _df.index[i] =='Canada' and _chem =='PFOA' and _c_type == C_fami[2]:
                    print(f'W_C_{_c_type}={_W_c} / {356}/65/{_pop} = {(_W_c)/(365*_pop*Weight)}')
                    
                _lis.append((_W_c)/(365*_pop*Weight))
                
            _df[(_family,_c_type)] = _lis
    
    _df.to_excel(excel_writer,sheet_name=_chem,index=True)
excel_writer.close()

100%|██████████| 10/10 [03:23<00:00, 20.33s/it]


In [35]:
#####################################
#####################################